In [35]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [36]:
# 미디어파이프 라이브러리 선언

mp_hands = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#데이터셋 directory 생성
os.makedirs('Datasets/', exist_ok=True)

In [45]:
#데이터셋 구성을 위해 400프레임간 촬영을 함.
study_frame=400 
#사용자의 손을 지정해줌.
hand="Right" 
#구역별 촬영간 5초의 준비시간을 줌.
pause_time=5
# 양치 구역을 알려줌. 
# U = UP, D = Down, R = Right, L = Left, F = Front, B = Back, O = occulusion (교합면)
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
#세션 지정
session=87

In [46]:
print('::Brushing teeth  Procedure::')
print(hand, end='\t')
# 양치 부위 순서를 알려줌.
for gesture in gesture_list:
    if gesture==gesture_list[-1]:
        print(gesture, end='')
    else:
        print(gesture, end='')
        print('->', end='')
print()
# 웹캠을 사용하여 영상을 찍어 좌표값을 추출함.
for gesture in gesture_list:
    cap = cv2.VideoCapture(0)

    position = np.zeros((1,21,3))
    velocity = np.zeros((1,21,3))
    acceleration = np.zeros((1,21,3))
# mediapipe 초기화
    with mp_hands.Hands(model_complexity=1,
                        max_num_hands=1,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5) as hands:



        print('::{} {} Ready.... '.format(hand,gesture))
        cnt=pause_time

        for i in [1000]*pause_time:
            print(cnt)
            cv2.waitKey(i)
            cnt-=1

        print('Start')
#지정한 study_frame만큼 촬영함.
        while len(position) <= study_frame:

            success, frame = cap.read()

            if not success:
                break

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                              mp_drawing_styles.get_default_hand_landmarks_style(),
                                              mp_drawing_styles.get_default_hand_connections_style())
#position을 구한다.
            frame_vector_x = []
            frame_vector_y = []
            frame_vector_z = []
            if results.multi_hand_landmarks:
                for point in results.multi_hand_landmarks[0].landmark:
                    frame_vector_x.append(point.x)
                    frame_vector_y.append(point.y)
                    frame_vector_z.append(point.z)
                frame_array = np.vstack((frame_vector_x,frame_vector_y,frame_vector_z)).reshape(1,21,3)

                position = np.vstack((position,frame_array))
#position을 이용해 velocity추출
                if len(position)>2:
                    current_velocity = position[-1] - position[-2]
                    velocity=np.vstack((velocity, current_velocity.reshape(1,21,3)))
#velocity를 이용해 accelation추출
                if len(position)>3:
                    current_acceleration = velocity[-1] - velocity[-2]
                    acceleration=np.vstack((acceleration, current_acceleration.reshape(1,21,3)))


            cv2.putText(frame,'Dataset Count : %d' % (position.shape[0]-1),(20,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))

            cv2.imshow('frame', frame)

            if cv2.waitKey(1) == ord('q'):
                break

        position=position[1:]
        velocity=velocity[1:]
        acceleration=acceleration[1:]

#position파일과 accelation파일을 numpy형태로 저장
        np.save('Datasets/%s_%s_p_%d' % (hand,gesture, session),position)
        np.save('Datasets/%s_%s_a_%d' % (hand,gesture, session),acceleration)
        cv2.waitKey(1000)


    cap.release()
    cv2.destroyAllWindows()
    print('Finished')

::Brushing teeth  Procedure::
Right	URF->UMF->ULF->DLF->DMF->DRF->URB->UMB->ULB->DLB->DMB->DRB->URO->ULO->DLO->DRO
::Right URF Ready.... 
2
1
Start
Finished
::Right UMF Ready.... 
2
1
Start
Finished
::Right ULF Ready.... 
2
1
Start
Finished
::Right DLF Ready.... 
2
1
Start
Finished
::Right DMF Ready.... 
2
1
Start
Finished
::Right DRF Ready.... 
2
1
Start
Finished
::Right URB Ready.... 
2
1
Start
Finished
::Right UMB Ready.... 
2
1
Start
Finished
::Right ULB Ready.... 
2
1
Start
Finished
::Right DLB Ready.... 
2
1
Start
Finished
::Right DMB Ready.... 
2
1
Start
Finished
::Right DRB Ready.... 
2
1
Start
Finished
::Right URO Ready.... 
2
1
Start
Finished
::Right ULO Ready.... 
2
1
Start
Finished
::Right DLO Ready.... 
2
1
Start
Finished
::Right DRO Ready.... 
2
1
Start
Finished
